<a href="https://colab.research.google.com/github/Maggiey01/Rights-Colab-YH/blob/main/DEF14A_DEI_explore_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# work pipeline
out of 37164/19349 proxy statements with 78 valid industry id
['EM-SV', 'TC-HW', 'RR-PP', nan, 'TC-TL', 'FN-AC', 'EM-MM', 'IF-EU',
       'FB-NB', 'FN-CB', 'TC-SI', 'SV-ED', 'HC-BP', 'RT-AE', 'EM-IS',
       'CG-MR', 'FB-RN', 'CG-HP', 'RT-IG', 'IF-RE', 'TC-SC', 'CG-BF',
       'CG-AA', 'TR-AP', 'EM-RM', 'HC-MS', 'RT-EE', 'SV-HL', 'SV-ME',
       'FB-FR', 'RT-CH', 'SV-PS', 'IF-WM', 'CG-EC', 'CG-AM', 'HC-MC',
       'SV-AD', 'IF-WU', 'TR-AF', 'FN-IN', 'FB-PF', 'FN-EX', 'RR-WT',
       'IF-EN', 'EM-MD', 'FB-AG', 'IF-RS', 'EM-EP', 'SV-CA', 'TR-RO',
       'RR-BI', 'RT-CP', 'FN-CF', 'FN-MF', 'FN-IB', 'HC-DY', 'TR-MT',
       'TC-IM', 'FB-TB', 'EM-CM', 'SV-LF', 'FB-MP', 'RR-ST', 'EM-CO',
       'IF-HB', 'TR-RA', 'CG-TS', 'TR-CR', 'HC-DI', 'TR-AL', 'TR-AU',
       'TR-CL', 'FB-AB', 'RR-FC', 'IF-GU', 'HC-DR', 'RR-FM', 'TC-ES']


1. create a industry_files dataframe
  1.1 load each year's txt folder to a dataframe  
  1.2 sort file directory alphabetically and each time loop 1000 files in a dataframe to avoid RAM limit  
  1.3 download and save each dataframe as csv to folder: https://drive.google.com/drive/folders/1YbP48lhJG3ATWr7vY7DJLiy8clgM9vVU  
  1.4 check whether csv columns collect comprehensive text from filings
- output: ['label', 'ticker', 'path', 'body', 'industry_id']


check each industries has at least 6 files

2. create comprehensive term regex cleaning Function 
attach regex to 
risk term/practice term/DEI context term ?
- simple
- complex

- output:
dict of dict
context_terms_regex dictionary\\for each
{DEI-context:
  {'DEI catergory1':
    {'extra_cleaning':F, 
    regex_lst:[compile+simple], 
    terms_to_remove:[]}}
  }，
}

3. create df - industry_all_events_dict
output:
dict of dict
industry_all_events_dict
{'industry_name1': industry_df1,
'industry_name2': industry_df2,
}

industry_df 
- keyword_ind columns: ?
DEI keywords,
work/employ root,
covid,
blm,

- practice/risk/dei term indicator columns: cleanterm_termtype_cat
e.g.
low wages_PRACTICE_Wages



all_industries_events_master_df = pd.concat(
    [industry_df for industry_name, industry_df in industry_all_events_dict.items()], ignore_index=True)

all_industries_events_master_df.shape

create a dictionary for ticker map industry id



### Import packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')
#"/content/drive/MyDrive/DFG/Proxy Statement/"

Mounted at /content/drive


In [2]:
import datetime
import numpy as np
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import random

import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from itertools import islice
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
#count files number
DEF_raw_dir = '/content/drive/MyDrive/DFG/Save DEF/'

def filenumber(rawpath):
  for y in ['2016', '2017', '2018', '2019', '2020', '2021']:
    path, dirs, files = next(os.walk(DEF_raw_dir+y+'/'))
    file_count = len(files)
    print(f'{file_count} files in {y} directory')

filenumber(DEF_raw_dir)


5973 files in 2016 directory
363 files in 2017 directory
5833 files in 2018 directory
5863 files in 2019 directory
6026 files in 2020 directory
6187 files in 2021 directory


In [4]:
def filenumber(rawpath):
  path, dirs, files = next(os.walk(rawpath))
  file_count = len(files)
  print(f'{file_count} files in {rawpath} directory')


### Create function to collect files text into dataframe
Input: path of each year files folder  
Read 1000 files from each year each time to put in a df  
Each row read 1 file  
Output: df
- Download each dataframe as csv


In [5]:
ind_map = pd.read_csv('/content/drive/MyDrive/DFG/usable_10ks_with_sics.csv')
ind_dict = pd.Series(ind_map.primary_industry_id.values,index=ind_map.ticker).to_dict()

In [6]:
def file_seeker(path_in, index_start, index_stop):
    import os
    import pandas as pd
    tmp_pd = pd.DataFrame()
    try:
      for count, fname in enumerate(islice(sorted(os.listdir(path_in)),index_start,index_stop)):
        if (fname[:4] != '_UNK') & (fname[-4:] != '.tsv'):
          tmp = open(path_in+fname, "r", encoding="ISO-8859-1")
          tmp_f = tmp.read()
          tmp.close()
          #paralist=[content.strip() for content in tmp_f.splitlines() if content]
          #for paraindex, paratext in enumerate(paralist):
          tmp_pd = tmp_pd.append({'label': fname,   
                                    'ticker': fname[:-29],                                 
                                    'path': path_in+fname, 
                                    'body': tmp_f}
                                   ,ignore_index=True)
        if count == index_stop:
          break
    except Exception as e:
      print (e)
      pass
    return tmp_pd, count

In [7]:
#count readable proxy statement after load into dataframe
def filedatalist(file_datalist):
  filesum=0
  for i in file_datalist:
    print(i.shape)
    filesum=filesum+i.shape[0]
  return filesum

### DEF 2016
5973 in all  
4323 after read  


In [ ]:
DEF16_raw_dir = '/content/drive/MyDrive/DFG/Save DEF/2016/'
DEF16_1, load_count = file_seeker(DEF16_raw_dir, 0, 1000)
DEF16_1['industry_id'] = DEF16_1['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF16_1.csv', 'w') as f:
  DEF16_1.to_csv(f)

In [ ]:
DEF16_2, load_count = file_seeker(DEF16_raw_dir, 1000, 2000)
DEF16_2['industry_id'] = DEF16_2['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF16_2.csv', 'w') as f:
  DEF16_2.to_csv(f)

In [ ]:
DEF16_3, load_count = file_seeker(DEF16_raw_dir, 2000, 3000)
DEF16_3['industry_id'] = DEF16_3['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF16_3.csv', 'w') as f:
  DEF16_3.to_csv(f)

In [ ]:
DEF16_4, load_count = file_seeker(DEF16_raw_dir, 3000, 4000)
DEF16_4['industry_id'] = DEF16_4['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF16_4.csv', 'w') as f:
  DEF16_4.to_csv(f)

In [ ]:
DEF16_5, load_count = file_seeker(DEF16_raw_dir, 4000, 5000)
DEF16_5['industry_id'] = DEF16_5['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF16_5.csv', 'w') as f:
  DEF16_5.to_csv(f)

In [ ]:
DEF16 = [DEF16_1,DEF16_2,DEF16_3,DEF16_4,DEF16_5]
filedatalist(DEF16)

(975, 5)
(1000, 5)
(1000, 5)
(1000, 5)
(348, 5)


4323

### DEF 2017???
363 files in /content/drive/MyDrive/DFG/Proxy Statement/2017/ directory

### DEF 2018
5833 in 2018 directory  
4734 after read 


In [8]:
DEF18_raw_dir = '/content/drive/MyDrive/DFG/Save DEF/2018/'
DEF18_1, load_count = file_seeker(DEF18_raw_dir, 0, 1000)
DEF18_1['industry_id'] = DEF18_1['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF18_1.csv', 'w') as f:
  DEF18_1.to_csv(f)

In [9]:
DEF18_2, load_count = file_seeker(DEF18_raw_dir, 1000, 2000)
DEF18_2['industry_id'] = DEF18_2['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF18_2.csv', 'w') as f:
  DEF18_2.to_csv(f)

In [10]:
DEF18_3, load_count = file_seeker(DEF18_raw_dir, 2000, 3000)
DEF18_3['industry_id'] = DEF18_3['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF18_3.csv', 'w') as f:
  DEF18_3.to_csv(f)

In [11]:
DEF18_4, load_count = file_seeker(DEF18_raw_dir, 3000, 4000)
DEF18_4['industry_id'] = DEF18_4['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF18_4.csv', 'w') as f:
  DEF18_4.to_csv(f)

In [12]:
DEF18_5, load_count = file_seeker(DEF18_raw_dir, 4000, 5000)
DEF18_5['industry_id'] = DEF18_5['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF18_5.csv', 'w') as f:
  DEF18_5.to_csv(f)

In [13]:
DEF18 = [DEF18_1,DEF18_2,DEF18_3,DEF18_4,DEF18_5]
filedatalist(DEF18)

(983, 5)
(1000, 5)
(1000, 5)
(1000, 5)
(751, 5)


4734

### DEF 2019
5863 in 2019 directory  
5017 after read



In [ ]:
DEF19_raw_dir = '/content/drive/MyDrive/DFG/Save DEF/2019/'
DEF19_1, load_count = file_seeker(DEF19_raw_dir, 0, 1000)
DEF19_1['industry_id'] = DEF19_1['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF19_1.csv', 'w') as f:
  DEF19_1.to_csv(f)

In [ ]:
DEF19_2, load_count = file_seeker(DEF19_raw_dir, 1000, 2000)
DEF19_2['industry_id'] = DEF19_2['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF19_2.csv', 'w') as f:
  DEF19_2.to_csv(f)

In [ ]:
DEF19_3, load_count = file_seeker(DEF19_raw_dir, 2000, 3000)
DEF19_3['industry_id'] = DEF19_3['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF19_3.csv', 'w') as f:
  DEF19_3.to_csv(f)

In [ ]:
DEF19_4, load_count = file_seeker(DEF19_raw_dir, 3000, 4000)
DEF19_4['industry_id'] = DEF19_4['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF19_4.csv', 'w') as f:
  DEF19_4.to_csv(f)

In [ ]:
DEF19_5, load_count = file_seeker(DEF19_raw_dir, 4000, 5000)
DEF19_5['industry_id'] = DEF19_5['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF19_5.csv', 'w') as f:
  DEF19_5.to_csv(f)

In [ ]:
DEF19_6, load_count = file_seeker(DEF19_raw_dir, 5000, 6000)
DEF19_6['industry_id'] = DEF19_6['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF19_6.csv', 'w') as f:
  DEF19_6.to_csv(f)

In [ ]:
DEF19 = [DEF19_1,DEF19_2,DEF19_3,DEF19_4,DEF19_5,DEF19_6]
filedatalist(DEF19)

(983, 5)
(1000, 5)
(1000, 5)
(1000, 5)
(1000, 5)
(34, 5)


5017

### DEF 2020
6026 in 2020 directory  
5362 after read

In [ ]:
DEF20_raw_dir = '/content/drive/MyDrive/DFG/Save DEF/2020/'
DEF20_1, load_count = file_seeker(DEF20_raw_dir, 0, 1000)
DEF20_1['industry_id'] = DEF20_1['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF20_1.csv', 'w') as f:
  DEF20_1.to_csv(f)

In [ ]:
DEF20_2, load_count = file_seeker(DEF20_raw_dir, 1000, 2000)
DEF20_2['industry_id'] = DEF20_2['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF20_2.csv', 'w') as f:
  DEF20_2.to_csv(f)

In [ ]:
DEF20_3, load_count = file_seeker(DEF20_raw_dir, 2000, 3000)
DEF20_3['industry_id'] = DEF20_3['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF20_3.csv', 'w') as f:
  DEF20_3.to_csv(f)

In [ ]:
DEF20_4, load_count = file_seeker(DEF20_raw_dir, 3000, 4000)
DEF20_4['industry_id'] = DEF20_4['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF20_4.csv', 'w') as f:
  DEF20_4.to_csv(f)

In [ ]:
DEF20_5, load_count = file_seeker(DEF20_raw_dir, 4000, 5000)
DEF20_5['industry_id'] = DEF20_5['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF20_5.csv', 'w') as f:
  DEF20_5.to_csv(f)

In [ ]:
DEF20_6, load_count = file_seeker(DEF20_raw_dir, 5000, 6000)
DEF20_6['industry_id'] = DEF20_6['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF20_6.csv', 'w') as f:
  DEF20_6.to_csv(f)

In [ ]:
DEF20 = [DEF20_1,DEF20_2,DEF20_3,DEF20_4,DEF20_5,DEF20_6]
filedatalist(DEF20)

(991, 5)
(1000, 5)
(1000, 5)
(1000, 5)
(1000, 5)
(371, 5)


5362

### DEF 2021
6187 files in 2021 directory  
5617 after read

In [ ]:
DEF21_raw_dir = '/content/drive/MyDrive/DFG/Save DEF/2021/'
DEF21_1, load_count = file_seeker(DEF21_raw_dir, 0, 1000)
DEF21_1['industry_id'] = DEF21_1['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF21_1.csv', 'w') as f:
  DEF21_1.to_csv(f)

In [ ]:
DEF21_2, load_count = file_seeker(DEF21_raw_dir, 1000, 2000)
DEF21_2['industry_id'] = DEF21_2['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF21_2.csv', 'w') as f:
  DEF21_2.to_csv(f)

In [ ]:
DEF21_3, load_count = file_seeker(DEF21_raw_dir, 2000, 3000)
DEF21_3['industry_id'] = DEF21_3['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF21_3.csv', 'w') as f:
  DEF21_3.to_csv(f)

In [ ]:
DEF21_4, load_count = file_seeker(DEF21_raw_dir, 3000, 4000)
DEF21_4['industry_id'] = DEF21_4['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF21_4.csv', 'w') as f:
  DEF21_4.to_csv(f)

In [ ]:
DEF21_5, load_count = file_seeker(DEF21_raw_dir, 4000, 5000)
DEF21_5['industry_id'] = DEF21_5['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF21_5.csv', 'w') as f:
  DEF21_5.to_csv(f)

In [ ]:
DEF21_6, load_count = file_seeker(DEF21_raw_dir, 5000, 6000)
DEF21_6['industry_id'] = DEF21_6['ticker'].apply(lambda x: ind_dict.get(x))
with open('/content/drive/My Drive/DFG/DEF14A_Data/DEF21_6.csv', 'w') as f:
  DEF21_6.to_csv(f)

In [ ]:
DEF21 = [DEF21_1,DEF21_2,DEF21_3,DEF21_4,DEF21_5,DEF21_6]
filedatalist(DEF21)

(995, 5)
(1000, 5)
(1000, 5)
(1000, 5)
(1000, 5)
(622, 5)


5617

## Check comprenshiveness
each dataframe['body'] collect comprehensive text from txt
- random generate 25 rows for each txt dataframe - collect 100 txt from 2018-2021 
- read ['body'] from csv?
- write in accordingly rows['path'] txt files, check 100 characters + bottom 100 characters
- compare with accordingly dataframe row

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [ ]:
def checktext(df_in, sample_number):
  #find accordingly csv
  csvpath = '/content/drive/My Drive/DFG/DEF14A_Data/'+get_df_name(df_in)+'.csv' 
  testcsv = pd.read_csv(csvpath)
  #random generate 25 rows from dataframe 
  sample_df = df_in.sample(sample_number)
  #make a index list to retreive that body text
  indexlist = list(sample_df.index)
  j=0 # check how many cell
  incompre = []
  for i in indexlist:   
    #text from csv
    csvtext = testcsv.iloc[i,:]['body']
    #text from df
    dftext = sample_df.iloc[j,:]['body']
    j=j+1
    if csvtext != dftext:
      incompre = incompre.append(i)
      print(f'raise {i}')
    
  print(f'check {j} sample')
  return incompre

In [ ]:
checktext(DEF18_1,25)

NameError: ignored

In [ ]:
checktext(DEF18_2,25)

sucessfully check 25 sample


25

In [ ]:
checktext(DEF18_3,25)

sucessfully check 25 sample


25

In [ ]:
checktext(DEF18_4,25)

sucessfully check 25 sample


25

In [ ]:
checktext(DEF18_5,25)

sucessfully check 25 sample


25

In [ ]:
checktext(DEF19_1,25)

sucessfully check 25 sample


25

In [ ]:
checktext(DEF19_2,25)

check 25 sample


[]

In [ ]:
checktext(DEF19_3,25)

check 25 sample


[]

In [ ]:
checktext(DEF19_4,25)

check 25 sample


[]

In [ ]:
checktext(DEF19_5,25)

check 25 sample


[]

In [ ]:
checktext(DEF19_6,25)

check 25 sample


[]

In [ ]:
checktext(DEF20_1,25)

check 25 sample


[]

In [ ]:
checktext(DEF20_2,25)

check 25 sample


[]

In [ ]:
#index = 839
testcsv = pd.read_csv('/content/drive/My Drive/DFG/DEF14A_Data/DEF20_2.csv')
sample_df = DEF20_2
testcsv.iloc[839,:]
sample_df.iloc[839,:]
#after checking, the original txt is empty

In [ ]:
checktext(DEF20_3,25)

check 25 sample


[]

In [ ]:
checktext(DEF20_4,25)

check 25 sample


[]

In [ ]:
checktext(DEF20_5,25)

check 25 sample


[]

In [ ]:
checktext(DEF20_6,25)

check 25 sample


[]

In [ ]:
checktext(DEF21_1,25)

check 25 sample


[]

In [ ]:
checktext(DEF21_2,25)

check 25 sample


[]

In [ ]:
checktext(DEF21_3,25)

check 25 sample


[]

In [ ]:
checktext(DEF21_4,25)

check 25 sample


[]

In [ ]:
checktext(DEF21_5,25)

check 25 sample


[]

In [ ]:
checktext(DEF21_6,25)

check 25 sample


[]

In [ ]:
def checktext2(df_ls, sample_number):
  for df_index,idf in enumerate(df_ls):
    #find accordingly csv
    csvpath = '/content/drive/My Drive/DFG/DEF14A_Data/'+get_df_name(idf)+'.csv' 
    testcsv = pd.read_csv(csvpath)
    #random generate 25 rows from dataframe 
    sample_df = idf.sample(sample_number)
    #make a index list to retreive that body text
    indexlist = list(sample_df.index)
    j=0 # check how many cell
    for i in indexlist:   
      #text from csv
      csvtext = testcsv.iloc[i,:]['body']
      #text from df
      dftext = sample_df.iloc[j,:]['body']
      j=j+1
      if csvtext != dftext:
        print(i+idf)
    
    print(f'sucessfully check {j} sample for {idf}')
  return df_index,j
  

In [ ]:
checktext2(DEF19,25)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:

#write in each file txt from that list

def readfile(pathlist):
  tmp_pd=pd.DataFrame()
  for index,i in enumerate(pathlist):
      
      tmp = open(i, "r", encoding="ISO-8859-1")
      tmp_f = tmp.read()
      tmp.close()

      tmp_pd = tmp_pd.append({'path': i,
                              'body': tmp_f}
                            ,ignore_index=True)
  return index, tmp_pd

index, tmp_pd =readfile(pathlist1)
tmp_pd.shape

(25, 2)

In [ ]:
from google.colab import files
df1.to_csv('exam1.csv') 
files.download("exam1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
industry_files = {'industry_name':['file.txt', 'file2.txt']}
industry_files['industry_name']

['file.txt', 'file2.txt']

In [ ]:
# check (list of industries)
industry_list = sorted(list(industry_files.keys()))

print(industry_list)
print("The number of industries:", len(industry_list))

['Advertising& Marketing', 'Aerospace& Defense', 'AgriculturalProducts', 'AirFreight & Logistics', 'Airlines', 'AlcoholicBeverages', 'Apparel,Accessories & Foo', 'ApplianceManufacturing', 'AssetManagement & Custody', 'AutoParts', 'Automobiles', 'Biofuels', 'Biotechnology& Pharmaceut', 'BuildingProducts & Furnis', 'CarRental & Leasing', 'Casinos& Gaming', 'Chemicals', 'CoalOperations', 'CommercialBanks', 'ConstructionMaterials', 'ConsumerFinance', 'Containers& Packaging', 'CruiseLines', 'DrugRetailers', 'E-Commerce', 'Education', 'ElectricUtilities & Power', 'Electrical& Electronic Eq', 'ElectronicManufacturing S', 'Engineering& Construction', 'FoodRetailers & Distribut', 'ForestryManagement', 'FuelCells & Industrial Ba', 'GasUtilities & Distributo', 'Hardware', 'HealthCare Delivery', 'HealthCare Distributors', 'HomeBuilders', 'Hotels& Lodging', 'Household& Personal Produ', 'IndustrialMachinery & Goo', 'Insurance', 'InternetMedia & Services', 'InvestmentBanking & Broke', 'Iron& Steel Pr

In [ ]:
# check that each industry has six file names
good_ind_filecount = 0

for industry in industry_files:

    if len(industry_files[industry]) != 6:
        print(industry, "does not have 6 files.")
    else:
        good_ind_filecount += 1
        
if good_ind_filecount == len(industry_files):
    print("All industries are good to go!")

All industries are good to go!


In [ ]:
def attach_regex_to_beginning_of_terms(terms_lst, regex='(?<![^ .,?!;])'):
    if regex == '(?<![^ .,?!;])':
        return [regex + term for term in terms_lst]
        
attach_regex_to_beginning_of_terms(['high turnover', 'worker turnover',])



['(?<![^ .,?!;])high turnover', '(?<![^ .,?!;])worker turnover']

## Create dictionary
Gather all practice terms and risk terms associated with labor practices

In [6]:
def attach_regex_to_beginning_of_terms(terms_lst, regex='(?<![^ .,?!;])'):
    if regex == '(?<![^ .,?!;])':
        return [regex + term for term in terms_lst]

# RISK term categories
risk_category_to_term_mapping_SIMPLE = {
    'Employee/Talent-Retention': [],

   'Worker-Protest': ['strike', 'sit-in', 'operational disruption',
                      'protest', 'injury'], 
   'Consumer-Protest': ['boycott', 'protest', 'social license'],
 
   'Operational-Costs': ['operational disruption', 'operating cost', 'delay',
                         'disruption'],
 
   'Financial-Loss': ['sanction', 
                      #'reimburse', 
                      'restitution', 'fine', 
              'penalt', 'bankrupt', 'liabl', ' financial loss', 
              #'lost'
              ],
 
   'Legal-Risk': ['lawsuit', 'litigation', 'impoundment', 'detain',
                  'penalt', 'sanction', 'court', 'consent decree'],
 
   'Reputational-Damage': ['brand damage', 'monetary damage',
                           'brand reputation', 'brand recognition',
                           'social license',
                           'decreased trust',
                           'decreased innovation',
                           'lost opportunit', 'resign'
                           ],
 
   "Remedy": [#'reimburse', 
              'divest', 'restitution', 'conciliation agreement'],
 
   "Modern-Slavery": ['modern slavery', 'debt bondage', 'human traffic'],
 
   'Other': ['alleg', 'accus', 'exploit', 'publicly expose', 'investigat',
            'enforcement', 'security force', 'inspection', 'inspector'],
 
   'Other-RK': ['scandal', 'government action', 'share price', 'share value',
                #'investment',
                # 'sales'
                ]
}
 
risk_category_to_term_mapping_COMPLEX = {
    'Employee/Talent-Retention': {
        'turnover': attach_regex_to_beginning_of_terms(['high turnover', 'worker turnover', 'employee turnover', 'turnover rate', 'voluntary turnover', 'quit rate']),
        'retention': attach_regex_to_beginning_of_terms(['low retention', 'retention rate', 'employee retention', 'worker retention']),
        'talent': attach_regex_to_beginning_of_terms(['attract talent', 'retain talent', 'find talent', 'talent acquisition']) 
    },
   'Worker-Protest': {'walkout': attach_regex_to_beginning_of_terms(['walk[- ]?out'])},
   'Consumer-Protest': {'social license': attach_regex_to_beginning_of_terms(['social licen[cs]e'])},
   'Operational-Costs': {'withhold release order': attach_regex_to_beginning_of_terms(['withhold release order', 'wro']),
                      'block import': attach_regex_to_beginning_of_terms(['block(.*)import', 'ban(.*)import', 'import(.*)ban',
                                                                          'prohibit(.*)import', 'import(.*)prohibit',
                                                                          'block(.*)entry', 'entry(.*)block',
                                                                          'seiz(.*)product', 'product(.*)seiz']),},
   'Financial-Loss': {'pay damages': attach_regex_to_beginning_of_terms(['pay(.*)damage', '(agreed|had|forced)(.*)to pay']),
             'seizure of assets': attach_regex_to_beginning_of_terms(['seiz(.*)asset'])},
   'Legal-Risk': {'court-ordered relief': attach_regex_to_beginning_of_terms(['monetary relief', 'equitable relief', 
                                                                              'injunctive relief', 'back pay', 
                                                                              'front pay', 'compensatory damages',
                                                                              '(punitive|exemplary) damages']),
                  'settlement': attach_regex_to_beginning_of_terms(['pay(.*)to settle'])
                                                                              },
   'Reputational-Damage': {'workplace shutdown': attach_regex_to_beginning_of_terms(['workplace shutdown', 'shutdown']),
                           'social license': attach_regex_to_beginning_of_terms(['social licen[cs]e'])},
   "Remedy": {},
   "Modern-Slavery": {
                      'forced labor': attach_regex_to_beginning_of_terms(['(forced|slave) labo[u]?r']),
                      'child labor': attach_regex_to_beginning_of_terms(['child labo[u]?r', 'child slave'])},
 
   'Other': {},
   'Other-RK': {'negative return': attach_regex_to_beginning_of_terms(['negative(.*)return'])}
}


# PRACTICE term categories UPDATED 11/29/2021 from Raiha's supply chain list
practice_category_to_term_mapping_SIMPLE = {
    #'Employee/Talent-Retention': [],
   'Wages': [
             'wage',
             'wage theft',
             'living wage'],
   'Precarious-Work': [
                       'precarity', 'precarious work',
                       'gig work',
                       'alternative work',
                       'alternate work',
                       'contingent work',
                       # 'migrant',
                       'informal work',
                       'casual work',
                       'hazardous work'
                       ],
   'Mdrn-Slav-Risk': [# 'broker', 'agent',
                      'confinement', 'document retention',
                      'restriction of movement',
                      'delayed wage',
                      'pay manipulation',
                      'punishment', 'poor food', #'retaliat',
                      # 'sexual violence', 'sexual harassment',
                      'deprivation'
                      'unpaid wage', 
				              'delayed payment',
                       'wage violation',
                       ],
 
   'Work-Conditions': [
                       # 'lockout',
                       'freedom of association',
                       'collective bargaining', 
                       'work stoppage',
                       'hotline',
                       #'worker retention'
                       ],
   'Good-Practices': ['code of conduct', 'due diligence',
                      'ethical recruit', # ethical recruitment
                      'handbook',  # supplier handbook
                      # 'supplier remediation',
                      'social audit',
                      # 'risk assessment',
                      'equal benefits',
                      'transparency', 'traceability', 'visibility', 'accessib'
                      # 'supply chain map',
                      'timely payments',
                      'union', 'worker committee', 
                      'empower', 'accommodati'
                      ],
                       
   'Neutral-Practices': [
                         # 'sourcing',
                         'outsourc', # Kept based on Joanne's feedback
                         # ‘raw material’,
                         'subcontracting',
                         # 'small-holder supply chain',
                         # 'overtime', 
                         # 'demand volatility',
                         'program', 'initiative', 'training', #'development',
                         'exempt', 'recruit', 'promotion', 'arbitration', 
                         'mentorship', 'affirmative action'
                         # investment (in people)
                         ],
   'Negative-Practices': [
                          # 'conflict',
                          'order delay',
                          'lead time', # previously 'short lead time'
                          'unplanned shipment',
                          'corruption',
                          # 'fraud',
                          'quota system', 'delayed payment',
                          'weak governance', 'wage violation',
                          'informal supply chain', 'last-minute order modification', 
                          'unfair timing demand', 'pricing pressure',
                          'poor forecasting', 'irresponsible exit'
                          ],
   'Other': []
}
 
practice_category_to_term_mapping_COMPLEX = {
   #'Employee/Talent-Retention': {
    #    'turnover': attach_regex_to_beginning_of_terms(['involuntary turnover', 'lay(.*)off']),
    #},

   'Wages': {# 'pricing': attach_regex_to_beginning_of_terms(['pricing', 'price']),
             'low wages': attach_regex_to_beginning_of_terms(['low wage', 'poverty-level wage']),
             'underpay': attach_regex_to_beginning_of_terms(['underpay', 'underpaid', 'inadequate pay', 'reduced pay']), 
             'DEI compensation': attach_regex_to_beginning_of_terms(['[un]?fair (pay|compensation)', '[un]?equal (pay|compensation)', 
                                                                     '(pay|wage|compensation) ([dis]?parit|bias|discrimination)',
                                                                     'pay equality', 'paid less', 'pay gap'])
             },
   'Precarious-Work': {'temporary work-employement': attach_regex_to_beginning_of_terms(['temporary( |-)(work|employ|contract)', 'non(-)?permanent (work|employ|contract)',
                                                                                         'seasonal( |-)(work|employ|contract)', 'part( |-)time worker']),
                       'contract labor': attach_regex_to_beginning_of_terms(['contract labo[u]?r ', 'contract work', 'contractor'])},
   'Mdrn-Slav-Risk': {
       # 'third party': attach_regex_to_beginning_of_terms(['third[- ]party'])
       'coercive labor': attach_regex_to_beginning_of_terms(['coercive labo[u]?r']),
       'prison labor': attach_regex_to_beginning_of_terms(['prison labo[u]?r']),
       'recruitment fee': attach_regex_to_beginning_of_terms(['recruitment(.*)fee']),
       'withhold wage': attach_regex_to_beginning_of_terms(['withh[oe]ld(ing)? wage']),
       'passport retention': attach_regex_to_beginning_of_terms(['passport retention', 'retention of passport', 'withh[oe]ld(ing)? passport'])
    },
    
    'Work-Conditions': {
        'collective bargaining agreement': attach_regex_to_beginning_of_terms(['collective bargaining agreement', 'cba']),
        'unsafe conditions': attach_regex_to_beginning_of_terms(['(toxic|hostile|poor|dire) work(ing| environment|place)', 'working conditions', 
                                                                 'dangerous', 'unsafe', 'unhealth', 'hazard', 'violence', 'assault']),
        'employee morale': attach_regex_to_beginning_of_terms(['employee morale', 'employee satisfaction']),
        'grievance mechanism': attach_regex_to_beginning_of_terms(['grievance mechanism', 'grievance system']),
        'reprisal/retaliation': attach_regex_to_beginning_of_terms(['retaliat', 'reprisal'])
    },
    
    'Good-Practices': {'code of conduct negative': attach_regex_to_beginning_of_terms(['code of conduct(.*)breach', 'breach(.*)code of conduct',
                                                                                    'violat(.*)code of conduct', 'code of conduct(.*)violat',
                                                                                    'non[- ]?compliance(.*)code of conduct', 'code of conduct(.*)non[- ]?compliance',
                                                                                    'break(.*)code of conduct',
                                                                                    'broken(.*)code of conduct', 'code of conduct(.*)broken',
                                                                                    'fail(.*)code of conduct']),
                      #'engagement': attach_regex_to_beginning_of_terms(['worker engagement', 'employee engagement']),
                      'flexible work': attach_regex_to_beginning_of_terms(['flexible work', 'flexible hour', 'remote( |-)work', 'work( |-)from( |-)home']),
                      'family leave': attach_regex_to_beginning_of_terms(['maternity leave', 'parental leave'])                                                         
   },
                      
                                             
   'Neutral-Practices': {'corrective action': attach_regex_to_beginning_of_terms(['corrective(.*)action', 'corrective(.*)plan', 'corrective(.*)measure']),
                         'hiring': attach_regex_to_beginning_of_terms(['hire', 'hiring', 'eligib', 'equal[ employment]? opportunit']),
                         'termination/layoff': attach_regex_to_beginning_of_terms(['termination','terminated','layoff','laid( |-)off', 'fired'])
                         },

   'Negative-Practices': {
       # 'piece work': attach_regex_to_beginning_of_terms(['piece work', 'piece[- ]rate']),
       # 'production target': attach_regex_to_beginning_of_terms(['production target', 'production quota']),
       'hour violation': attach_regex_to_beginning_of_terms(['hour (law )?violation']),
       'canceled order': attach_regex_to_beginning_of_terms(['cancel[l]?ed order']),
       'overtime NEGATIVE': attach_regex_to_beginning_of_terms(['(forced|unpaid|chronic|mandatory|unlawful) overtime']),
       'lead time NEGATIVE': attach_regex_to_beginning_of_terms(['short lead time', 'inadequate lead time'])},

   'Other': {}
}

In [7]:
# DEI Context Terms
DEI_context_category_to_term_mapping_SIMPLE = {
    'DEI-Context': ['ethnic', 'sexual', 'disabili',
                    'marital status', 'working mother', 'pregnant',
                    'veteran', 'bias', 'religio'
                    ],
    'DEI-Standalone-Terms': ['discriminat', 'harass']
}

DEI_context_category_to_term_mapping_COMPLEX = {
    'DEI-Context': {'race': attach_regex_to_beginning_of_terms(['race', 'racism', 'racist', 'racial', 'bipoc', 'people of colo[u]?r', 'blackface']),
                    'familial status': attach_regex_to_beginning_of_terms(['working (famil|parent|mother)']), 
                    'minorit': attach_regex_to_beginning_of_terms(['minorities', 'minority group']),
                    'LGBT': attach_regex_to_beginning_of_terms(['lgbt', 'sexual orientation', 'gender identity', 'gay', 'lesbian', 
                                                                'bisexual', 'transgender', 'queer', 'asexual', 'homophobia', 'nonbinary']),
                    'gender-M/F': attach_regex_to_beginning_of_terms(['female' ,'gender','working mother', 'based on sex'
                                                                      'pregnant', 'on the basis of sex', 'maternity leave', 
                                                                      'sexist', 'sex discrimination']),
                    'age': attach_regex_to_beginning_of_terms(['age discrimin', 'age bias', 'ageism', 'average age']),
                    # 'justice': attach_regex_to_beginning_of_terms(['racial justice', 'social justice']),
                    # 'equity': attach_regex_to_beginning_of_terms(['racial [in]?equity', 'gender [in]?equity', 'social [in]?equity', '[in]?equitable']),
                    'engagement': attach_regex_to_beginning_of_terms(['worker engagement', 'employee engagement']),
                    'equality': attach_regex_to_beginning_of_terms(['racial [in]?equality', 'gender [in]?equality', 'social [in]?equality']),
                    'nationality': attach_regex_to_beginning_of_terms(['nationality', 'national origin', 'foreign nationals']),
                    'underrepresented': attach_regex_to_beginning_of_terms(['under[-]?represented']),
                    'migrant': attach_regex_to_beginning_of_terms(['[im]?migrant', 'foreigner', 'visa', 'citizen', 'foreign worker']),
                    'education/skill level': attach_regex_to_beginning_of_terms(['entry(.*)level','education level',
                                                                                 'college degree', 'undergraduate degree', 
                                                                                 'graduate degree', 'high school diploma',
                                                                                 'low(-| )skill', 'high(-|ly | )skill']),
                    'economic status': attach_regex_to_beginning_of_terms(['economic status', 'economic class', 'low(.*)income', #'high(.*)income',
                                                                          'impoverish', 'poverty', 'middle class', 'working class']),
                    'criminal history': attach_regex_to_beginning_of_terms(['criminal history', 'felon', 'background check', 'convict']),
                    #'precarious worker': attach_regex_to_beginning_of_terms(['temporary(.*)worker', 'temporary(.*)employee', 'contract labo[u]?rer',
                     #                                                        'contract worker', 'contractor', 'seasonal(.*)employee', 'seasonal(.*)worker',
                     #                                                        'part(.*)time worker', 'part(.*)time employee']),
                    'DEI abroad-factory': attach_regex_to_beginning_of_terms(['factory work']),},
      
      'DEI-Standalone-Terms': {
          #'profiled': ['racially profile', 'racial profile']
          }
}

In [8]:
# term_type_dict = {'term_type_cat': {'clean term': 'regex_lst': {['fmt1', 'fmt2']}, 'extra_cleaning': True}}

def create_comprehensive_term_regex_cleaning_dict(
    term_type, category_to_term_mapping_SIMPLE, category_to_term_mapping_COMPLEX):

    term_type_regexes_cleaning = {}
    for term_cat, term_lst in category_to_term_mapping_SIMPLE.items():

        # Create dictionary for term type category
        if term_cat not in term_type_regexes_cleaning:
            term_type_regexes_cleaning[term_cat] = dict()
        
        # Variable for new category dict for `term_type_regexes_cleaning`
        term_cat_dict = term_type_regexes_cleaning[term_cat]

        term_cat_SIMPLE_lst = category_to_term_mapping_SIMPLE[term_cat]
        for term_SIMPLE in term_cat_SIMPLE_lst:
            # term_regex = re.compile(attach_regex_to_beginning_of_terms([term_SIMPLE])[0])
            term_regex_str_lst = attach_regex_to_beginning_of_terms([term_SIMPLE])
            term_cat_dict[term_SIMPLE] = {'regex_lst': term_regex_str_lst, 'extra_cleaning': False, 'terms_to_remove': []}  # TODO: Create detailed function for examples of the term to ignore, wich will have extra_cleaning=True

        term_cat_COMPLEX = category_to_term_mapping_COMPLEX[term_cat]
        for term_clean_COMPLEX, term_COMPLEX_regex_list in term_cat_COMPLEX.items():
            # term_regex_str_lst = attach_regex_to_beginning_of_terms(term_COMPLEX_dict)
            term_cat_dict[term_clean_COMPLEX] = {'regex_lst': [re.compile(regex) for regex in term_COMPLEX_regex_list], 'extra_cleaning': False, 'terms_to_remove': []} 

        term_type_regexes_cleaning[term_cat] = term_cat_dict

    return term_type_regexes_cleaning

In [9]:
DEI_context_terms_regex_dict = create_comprehensive_term_regex_cleaning_dict(
    term_type='DEI-context', 
    category_to_term_mapping_SIMPLE=DEI_context_category_to_term_mapping_SIMPLE, 
    category_to_term_mapping_COMPLEX=DEI_context_category_to_term_mapping_COMPLEX)

practice_terms_regex_dict = create_comprehensive_term_regex_cleaning_dict(
    term_type='practice', 
    category_to_term_mapping_SIMPLE=practice_category_to_term_mapping_SIMPLE, 
    category_to_term_mapping_COMPLEX=practice_category_to_term_mapping_COMPLEX)

risk_terms_regex_dict = create_comprehensive_term_regex_cleaning_dict(
    term_type='risk', 
    category_to_term_mapping_SIMPLE=risk_category_to_term_mapping_SIMPLE, 
    category_to_term_mapping_COMPLEX=risk_category_to_term_mapping_COMPLEX)

# Adding extra cleaning terms here (during processing, any mentions of 
# practice/risk terms in the irrelevant contexts below will be excluded)

# race
DEI_context_terms_regex_dict['DEI-Context']['race']['extra_cleaning'] = True
DEI_context_terms_regex_dict['DEI-Context']['race']['terms_to_remove'] = ['arms race', 'a race to', 'the race to', 'rat race', 'to corrosive racial inequality']

# turnover
risk_terms_regex_dict['Employee/Talent-Retention']['turnover']['extra_cleaning'] = True
risk_terms_regex_dict['Employee/Talent-Retention']['turnover']['terms_to_remove'] = ['involuntary turnover']

# shutdown
risk_terms_regex_dict['Reputational-Damage']['workplace shutdown']['extra_cleaning'] = True
risk_terms_regex_dict['Reputational-Damage']['workplace shutdown']['terms_to_remove'] = ['year-end shutdown','maintenance shutdown']

# fine
risk_terms_regex_dict['Financial-Loss']['fine']['extra_cleaning'] = True
risk_terms_regex_dict['Financial-Loss']['fine']['terms_to_remove'] = ['finecast', 'fine fragrance', 'fine and tall', 'driftable fine', 'fine chemical', 'fine construction level', 'fine-grain', 'fine partic', 'finergreen', 'fine molecular', 'fine wine', 'fine product', 'fine paper', 'fine balanc', 'fine-tun', 'fine fib', 'finely', 'fine, sand', 'fine tea', 'finest', 'finesse']
 
# strike
risk_terms_regex_dict['Worker-Protest']['strike']['extra_cleaning'] = True
risk_terms_regex_dict['Worker-Protest']['strike']['terms_to_remove'] = [r'disaster(s\b|\b) strike', 'strike deal', 'weather strike', 'strike a conversation']

# union
practice_terms_regex_dict['Good-Practices']['union']['extra_cleaning'] = True
practice_terms_regex_dict['Good-Practices']['union']['terms_to_remove'] = ['european union', 'customs union', 'american civil liberties union']
 
# pricing pressure
practice_terms_regex_dict['Negative-Practices']['pricing pressure']['context_words'] = ['supplier', 'factory', 'manufactur']


## Apply labor-practice indicators and practice term/risk term indicators to events
- Column "**marked_labor_relevant_ind**" is the final indicator
- One indicator column is created for each practice/risk term

drop na rows

drop duplicate col?

- Apply DEI (2/3) heuristic
- Apply work/employ root word check
- Apply COVID heuristic
- Apply Black Lives Matter heuristic

- PRACTICE TERM INDICATORS CREATED HERE
- RISK TERM INDICATORS CREATED HERE
- DEI TERM INDICATORS CREATED HERE

group by industry_id

In [10]:
DEF18_1df = pd.read_csv('/content/drive/My Drive/DFG/DEF14A_Data/DEF18_1.csv')
# extract files with industry id
DEF18_1df = DEF18_1df[~DEF18_1df['industry_id'].isna()]
#661 after selection

In [13]:
DEF18_1df.shape

(661, 6)

In [12]:
#drop na
DEF18_1df.dropna(axis=0, how='all', inplace=True)

In [15]:
DEF18_1df['industry_id'].unique()

array(['TR-AL', 'FN-AC', 'FN-IN', 'TC-SC', 'RT-EE', 'CG-MR', 'IF-RE',
       'TR-AF', 'EM-MM', 'HC-BP', 'FN-CB', 'HC-MS', 'FN-MF', 'TC-SI',
       'HC-DY', 'SV-PS', 'CG-HP', 'RT-IG', 'FB-AG', 'TC-HW', 'IF-EU',
       'CG-AA', 'EM-RM', 'CG-BF', 'RT-CH', 'SV-LF', 'IF-EN', 'TC-IM',
       'RT-AE', 'TC-TL', 'FN-CF', 'TC-ES', 'TR-AP', 'EM-EP', 'SV-ME',
       'FN-IB', 'RR-WT', 'RR-BI', 'CG-EC', 'HC-MC', 'EM-CM', 'TR-RO',
       'EM-CO', 'SV-ED', 'RR-ST', 'EM-IS', 'RT-CP', 'IF-WU', 'IF-WM',
       'FB-RN', 'EM-MD', 'FB-PF', 'FB-NB', 'FB-MP', 'SV-CA', 'HC-DI',
       'IF-RS', 'TR-CR', 'FB-FR', 'RR-FC', 'FN-EX', 'TR-CL', 'SV-AD',
       'IF-HB', 'SV-HL', 'CG-TS', 'EM-SV', 'IF-GU', 'RR-PP'], dtype=object)

In [16]:
# covid_lp_relevant_keywords = ['block', 'review', 'police', 'issue', r'licen[cs]e', 'wage', 'revoke', 'temporary'] as of 10/11
# pattern_covid_lp_relevant_keywords = '|'.join(covid_lp_relevant_keywords)

# Adaptation of Raiha's supply chain labor practice heuristic - get_labor_indicator_v2  
def get_DEI_indicator_v2(row, DEI_keywords): 
    """Applying this heuristic to generate indicator for mentions of 
    at least 2 of the following DEI keywords: 
    ['diversity', 'engage', 'inclusi']. 
    (v1 included all words as well as "employee".) """
    cols = [f'{keyword}_ind' for keyword in DEI_keywords]
    count_DEI_term_inds_1 = 0
    for col in cols:
        if row[col] == 1:
            count_DEI_term_inds_1 += 1
    
    return count_DEI_term_inds_1
    '''
    if count_DEI_term_inds_1 >= 2:
        return 1
    else:
        return 0
    '''

In [17]:
# DEI heuristic (flag if 2/4 terms are found)
DEI_keywords = ['diversity', 'equity', 'inclusion', ' dei'] # replaced employee engage

# labor_keywords = [r'labo[u]r', 'wage', 'worker']
covid_keywords = ['covid', 'coronavirus', 'pandemic']
blm_keywords = ['blm', 'black lives matter', 'george floyd']
pattern_DEI = '|'.join(DEI_keywords)

# Apply DEI (2/3) heuristic
for keyword in DEI_keywords:
  keyword_regex = re.compile(f'(?<![^ .,?!;]){keyword}')
  DEF18_1df[f'{keyword}_ind'] = np.where(DEF18_1df['body'].str.contains(keyword_regex) | DEF18_1df['body'].str.contains(keyword_regex), 
                                                  1, 0)
  #DEF18_1df['DEI_keyword_ind'] = DEF18_1df.apply(lambda row: get_DEI_indicator_v2(row, DEI_keywords), axis=1)
  #industry_df['marked_DEI_relevant_ind'] = industry_df['DEI_keyword_ind']

In [37]:
DEF18_1df['DEI_keyword_ind'] = DEF18_1df.apply(lambda row: get_DEI_indicator_v2(row, DEI_keywords), axis=1)

In [38]:
DEF18_1df.columns

Index(['Unnamed: 0', 'label', 'ticker', 'path', 'body', 'industry_id',
       'diversity_ind', 'equity_ind', 'inclusion_ind', ' dei_ind',
       'DEI_keyword_ind'],
      dtype='object')

In [43]:
DEF18_1df[['industry_id',
       'diversity_ind', 'equity_ind', 'inclusion_ind', ' dei_ind',
       'DEI_keyword_ind']].head(5)

,industry_id,diversity_ind,equity_ind,inclusion_ind,dei_ind,DEI_keyword_ind
5,TR-AL,1,1,1,0,3
6,FN-AC,1,1,0,0,2
7,FN-IN,0,0,0,0,0
8,TC-SC,1,1,1,0,3
9,RT-EE,1,1,0,0,2


In [19]:
# Apply work/employ root word check
root_employ = ['work', 'employ']
for keyword in root_employ:
  keyword_regex = re.compile(f'(?<![^ .,?!;]){keyword}')
  DEF18_1df[f'{keyword}_ind'] = np.where(DEF18_1df['body'].str.contains(keyword_regex) | DEF18_1df['body'].str.contains(keyword_regex), 
                                                  1, 0)
DEF18_1df['root_employ_ind'] = DEF18_1df.apply(lambda row: get_DEI_indicator_v2(row, root_employ), axis=1)
      #industry_df['marked_root_employ_ind'] = industry_df['root_employ_ind']

In [20]:
DEF18_1df.columns

Index(['Unnamed: 0', 'label', 'ticker', 'path', 'body', 'industry_id',
       'diversity_ind', 'equity_ind', 'inclusion_ind', ' dei_ind', 'work_ind',
       'employ_ind', 'root_employ_ind'],
      dtype='object')

In [21]:
# PRACTICE TERM INDICATORS CREATED HERE
term_type = 'practice'
for cat, cat_terms in practice_terms_regex_dict.items():
  for clean_term, term_dict in cat_terms.items():
    indicator_col_name = "{}_{}_{}".format(clean_term, term_type.upper(), cat)
    keyword_regexes = term_dict['regex_lst']
    DEF18_1df[indicator_col_name] = 0
    for keyword_regex_str in keyword_regexes:
      keyword_regex = re.compile(keyword_regex_str)
      DEF18_1df[indicator_col_name] = np.where(DEF18_1df['body'].str.contains(keyword_regex) | DEF18_1df['body'].str.contains(keyword_regex), 1, DEF18_1df[indicator_col_name])
      if term_dict['extra_cleaning']:
        pattern_terms_to_remove = re.compile('|'.join(term_dict['terms_to_remove']))
        DEF18_1df[indicator_col_name] = np.where(DEF18_1df['body'].str.contains(pattern_terms_to_remove) | DEF18_1df['body'].str.contains(pattern_terms_to_remove), 0, DEF18_1df[indicator_col_name])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.


KeyboardInterrupt: ignored

In [ ]:
DEF18_1df.columns
#took 15m

## Apply part 2


df  
add indicators of any_practice, any_risk?  
any practice/risk terms cooccurence  

all_industries_events_master_df['PRACTICE_TERMS_FOUND'] = "None"
all_industries_events_master_df['RISK_TERMS_FOUND'] = "None"
all_industries_events_master_df['DEI-CONTEXT_TERMS_FOUND'] = "None"
all_industries_events_master_df['DEI_TERMS_FOUND'] = "None"

In [24]:
DEF18_1df.columns

Index(['Unnamed: 0', 'label', 'ticker', 'path', 'body', 'industry_id',
       'diversity_ind', 'equity_ind', 'inclusion_ind', ' dei_ind', 'work_ind',
       'employ_ind', 'root_employ_ind', 'wage_PRACTICE_Wages',
       'wage theft_PRACTICE_Wages', 'living wage_PRACTICE_Wages',
       'low wages_PRACTICE_Wages', 'underpay_PRACTICE_Wages',
       'DEI compensation_PRACTICE_Wages', 'precarity_PRACTICE_Precarious-Work',
       'precarious work_PRACTICE_Precarious-Work',
       'gig work_PRACTICE_Precarious-Work',
       'alternative work_PRACTICE_Precarious-Work',
       'alternate work_PRACTICE_Precarious-Work',
       'contingent work_PRACTICE_Precarious-Work',
       'informal work_PRACTICE_Precarious-Work',
       'casual work_PRACTICE_Precarious-Work',
       'hazardous work_PRACTICE_Precarious-Work',
       'temporary work-employement_PRACTICE_Precarious-Work',
       'contract labor_PRACTICE_Precarious-Work',
       'confinement_PRACTICE_Mdrn-Slav-Risk',
       'document retention_

In [31]:
DEF18_1df.groupby('industry_id').count()

,Unnamed: 0,label,ticker,path,body,diversity_ind,equity_ind,inclusion_ind,dei_ind,work_ind,employ_ind,root_employ_ind,wage_PRACTICE_Wages,wage theft_PRACTICE_Wages,living wage_PRACTICE_Wages,low wages_PRACTICE_Wages,underpay_PRACTICE_Wages,DEI compensation_PRACTICE_Wages,precarity_PRACTICE_Precarious-Work,precarious work_PRACTICE_Precarious-Work,gig work_PRACTICE_Precarious-Work,alternative work_PRACTICE_Precarious-Work,alternate work_PRACTICE_Precarious-Work,contingent work_PRACTICE_Precarious-Work,informal work_PRACTICE_Precarious-Work,casual work_PRACTICE_Precarious-Work,hazardous work_PRACTICE_Precarious-Work,temporary work-employement_PRACTICE_Precarious-Work,contract labor_PRACTICE_Precarious-Work,confinement_PRACTICE_Mdrn-Slav-Risk,document retention_PRACTICE_Mdrn-Slav-Risk,restriction of movement_PRACTICE_Mdrn-Slav-Risk,delayed wage_PRACTICE_Mdrn-Slav-Risk,pay manipulation_PRACTICE_Mdrn-Slav-Risk,punishment_PRACTICE_Mdrn-Slav-Risk,poor food_PRACTICE_Mdrn-Slav-Risk,deprivationunpaid wage_PRACTICE_Mdrn-Slav-Risk,delayed payment_PRACTICE_Mdrn-Slav-Risk,wage violation_PRACTICE_Mdrn-Slav-Risk,coercive labor_PRACTICE_Mdrn-Slav-Risk,prison labor_PRACTICE_Mdrn-Slav-Risk,recruitment fee_PRACTICE_Mdrn-Slav-Risk,withhold wage_PRACTICE_Mdrn-Slav-Risk,passport retention_PRACTICE_Mdrn-Slav-Risk,freedom of association_PRACTICE_Work-Conditions,collective bargaining_PRACTICE_Work-Conditions,work stoppage_PRACTICE_Work-Conditions,hotline_PRACTICE_Work-Conditions,collective bargaining agreement_PRACTICE_Work-Conditions,unsafe conditions_PRACTICE_Work-Conditions,employee morale_PRACTICE_Work-Conditions,grievance mechanism_PRACTICE_Work-Conditions,reprisal/retaliation_PRACTICE_Work-Conditions,code of conduct_PRACTICE_Good-Practices,due diligence_PRACTICE_Good-Practices,ethical recruit_PRACTICE_Good-Practices,handbook_PRACTICE_Good-Practices,social audit_PRACTICE_Good-Practices,equal benefits_PRACTICE_Good-Practices,transparency_PRACTICE_Good-Practices,traceability_PRACTICE_Good-Practices,visibility_PRACTICE_Good-Practices,accessibtimely payments_PRACTICE_Good-Practices,union_PRACTICE_Good-Practices,worker committee_PRACTICE_Good-Practices,empower_PRACTICE_Good-Practices,accommodati_PRACTICE_Good-Practices,code of conduct negative_PRACTICE_Good-Practices,flexible work_PRACTICE_Good-Practices,family leave_PRACTICE_Good-Practices,outsourc_PRACTICE_Neutral-Practices,subcontracting_PRACTICE_Neutral-Practices,program_PRACTICE_Neutral-Practices,initiative_PRACTICE_Neutral-Practices,training_PRACTICE_Neutral-Practices,exempt_PRACTICE_Neutral-Practices,recruit_PRACTICE_Neutral-Practices,promotion_PRACTICE_Neutral-Practices,arbitration_PRACTICE_Neutral-Practices,mentorship_PRACTICE_Neutral-Practices,affirmative action_PRACTICE_Neutral-Practices,corrective action_PRACTICE_Neutral-Practices,hiring_PRACTICE_Neutral-Practices,ANY_DEI-CONTEXT_TERM,ANY_PRACTICE_TERM,ANY_RISK_TERM,ANY_DEI_TERM
industry_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CG-AA,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
CG-BF,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
CG-EC,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
CG-HP,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
CG-MR,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,

In [23]:
# Adding indicators of having any practice term or risk term, 
# to quickly identify events with co-occurrences of practice and risk terms

# indicator format: {term}_{PRACTICE/RISK/DEI-context term}_{category}

all_cols = list(DEF18_1df.columns)
practice_term_cols = []
risk_term_cols = []
DEI_context_term_cols = []
DEI_term_cols = []

for col in all_cols: 
    
    try:
        col_type = col.split('_')[1]
        DEI_col_list = [d+'_ind' for d in DEI_keywords]

        if col_type == 'PRACTICE':
            practice_term_cols.append(col)
        elif col_type == 'RISK':
            risk_term_cols.append(col)
        #elif col_type == 'DEI_context'.upper():
        elif col_type == 'DEI-CONTEXT':
            DEI_context_term_cols.append(col)
        elif col in DEI_col_list:
            DEI_term_cols.append(col)
    except:
        continue
                                     

In [25]:
DEF18_1df['ANY_DEI-CONTEXT_TERM'] = 0
for DEI_context_term_col in DEI_context_term_cols:
    DEF18_1df['ANY_DEI-CONTEXT_TERM'] = np.where(DEF18_1df[DEI_context_term_col] == 1, 
                                                                    1, DEF18_1df['ANY_DEI-CONTEXT_TERM'])
    

In [27]:
DEF18_1df['ANY_DEI-CONTEXT_TERM'].unique()

array([0])

In [28]:
DEF18_1df['ANY_DEI-CONTEXT_TERM'] = 0
for DEI_context_term_col in DEI_context_term_cols:
    DEF18_1df['ANY_DEI-CONTEXT_TERM'] = np.where(DEF18_1df[DEI_context_term_col] == 1, 
                                                                    1, DEF18_1df['ANY_DEI-CONTEXT_TERM'])
    
DEF18_1df['ANY_PRACTICE_TERM'] = 0
for practice_term_col in practice_term_cols:
    DEF18_1df['ANY_PRACTICE_TERM'] = np.where(DEF18_1df[practice_term_col] == 1, 
                                                                    1, DEF18_1df['ANY_PRACTICE_TERM'])
    
DEF18_1df['ANY_RISK_TERM'] = 0
for risk_term_col in risk_term_cols:
    DEF18_1df['ANY_RISK_TERM'] = np.where(DEF18_1df[risk_term_col] == 1, 
                                                                    1, DEF18_1df['ANY_RISK_TERM'])

DEF18_1df['ANY_DEI_TERM'] = 0
#DEF18_1df['ANY_DEI_TERM'] = np.where(DEF18_1df['DEI_keyword_ind'] >= 2, 
#                                                                1, DEF18_1df['ANY_DEI_TERM'])

DEF18_1df['ANY_ROOT_EMPLOY_TERM'] = 0
DEF18_1df['ANY_ROOT_EMPLOY_TERM'] = np.where(DEF18_1df['root_employ_ind'] >= 1, 
                                                                1, DEF18_1df['ANY_ROOT_EMPLOY_TERM'])

DEF18_1df['ANY_COVID_TERM'] = 0
DEF18_1df['ANY_COVID_TERM'] = np.where(DEF18_1df['COVID_keyword_ind'] >= 1, 
                                                                1, DEF18_1df['ANY_COVID_TERM'])
DEF18_1df['ANY_BLM_TERM'] = 0
DEF18_1df['ANY_BLM_TERM'] = np.where(DEF18_1df['BLM_keyword_ind'] >= 1, 
                                                                1, DEF18_1df['ANY_BLM_TERM'])


KeyError: ignored